In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

In [9]:
df = pd.read_csv('martial-law-narratives.csv')
df.head()

,url
0,https://www.martiallawchroniclesproject.com/bo...
1,https://www.martiallawchroniclesproject.com/wo...
2,https://www.martiallawchroniclesproject.com/wo...
3,http://www.positivelyfilipino.com/magazine/mar...
4,https://www.martiallawchroniclesproject.com/ma...


In [10]:
def read_urls(file_name):
    df = pd.read_csv(file_name)
    df[['title', 'time', 'narrative']] = df.apply(_parse_and_extract, axis = 1)
    return df

In [11]:
def _parse_and_extract(row):
    url = row['url']
    headers = {'User-Agent':'Mozilla/5.0'}
    r = requests.get(url, headers = headers)
    narrative_list = []
    soup = BeautifulSoup(r.content, 'html.parser')

    if soup.find('h1') == None:
        title = 'No Title'
    else:
        title = soup.find('h1').get_text()
    time = ' '.join(str(soup.find(id="art_plat")).split()[-5:])[:-6]
    texts = soup.findAll('p')
    for text in texts:
        narrative_list.append(text.get_text())

    narrative = '\n'.join(narrative_list)
    return pd.Series([title, time, narrative])

In [12]:
df = read_urls('martial-law-narratives.csv')

In [13]:
df.head()

,url,title,time,narrative
0,https://www.martiallawchroniclesproject.com/bo...,"Body blows, electric shock, bottle taps",,"by MLC | Aug 12, 2020 | Martial Law Stories\n..."
1,https://www.martiallawchroniclesproject.com/wo...,Women victims tell SC of Martial Law torture,,"by MLC | Aug 31, 2016 | Torture, Rape | 0 com..."
2,https://www.martiallawchroniclesproject.com/wo...,Women victims tell SC of Martial Law torture,,"by MLC | Aug 31, 2016 | Torture, Rape | 0 com..."
3,http://www.positivelyfilipino.com/magazine/mar...,The Magazine,,POSITIVELY FILIPINO is the premier digital nat...
4,https://www.martiallawchroniclesproject.com/ma...,MARTIAL LAW VICTIMS | ‘I didn’t eat for a week...,,"by MLC | Sep 17, 2012 | Torture | 0 comments\..."


In [14]:
df.to_csv("martial-law-narratives.csv", index=False)